<a href="https://colab.research.google.com/github/JeanMusenga/PhD-Thesis_2024_Musenga/blob/main/Classifiers_with_BoW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Important and useful sources:
https://stackoverflow.com/questions/61631446/attributeerror-list-object-has-no-attribute-lower-with-countvectorizer

In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import nltk
import gensim
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [7]:
np.random.seed(42) #The random seed helps in the reproduction of the results

In [10]:
dataset=pd.read_excel('posts.xlsx')

In [11]:
dataset.head()

,Question_body,Label
0,Here's the architecture:\n<ol>\n<li>Input comp...,1
1,I am confused about the architecture of Razor ...,1
2,I have some spring boot microservices and I wa...,1
3,I'm trying to properly design an application a...,1
4,I heard that for .NET8 Microsoft gifted us wit...,1


In [12]:
dataset.shape #I have 14918 instances/posts and 2 attributes/columns in the dataset.

(14932, 2)

In [15]:
dataset.Label.head()

,Label
0,1
1,1
2,1
3,1
4,1


In [ ]:
dataset.columns

Index(['Post', 'Class'], dtype='object')

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14932 entries, 0 to 14931
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Post    14932 non-null  object
 1   Class   14932 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 233.4+ KB


In [ ]:
dataset[dataset['Class'] ==1].count()

Post     7466
Class    7466
dtype: int64

In [ ]:
dataset[dataset['Class'] ==0].count()

Post     7466
Class    7466
dtype: int64

In [ ]:
dataset.Post[304]

"<p>I am new to microservices , and currently designing a SASS based multi tenant application .\nnow i have 3 services , one for auth related activities , one for user ,roles and rbac and one for tenant  all the services have their own respective DB's.\nnow since its a multi tenant application. i am not able to decide in which DB do i store the tenant - User related mapping .\ndo i create a seperate tables for mapping tenets and store in the tenant service DB or do i take the tenant ID from the service and store the tenant related mapping in the user service Database .</p>\n<p>Thanks .</p>\n"

# Tokenization

In [16]:
import spacy
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)
print("-----------------------------------------------")

import string
punctuations = string.punctuation
print(punctuations)

{'various', 'another', "'re", 'fifteen', 'a', 'doing', 'nobody', 'nowhere', 'of', 'during', 'forty', 'put', 'first', 'take', 'nine', 'towards', 'formerly', 'also', 'some', 'his', 'yourselves', 'meanwhile', 'almost', 'upon', 'it', 'onto', 'do', '‘s', 'that', 'will', 'among', 'before', 'anything', 'see', 'give', 'many', 'amount', 'or', 'everywhere', 'they', 'anyway', 'how', 'everything', 'otherwise', 'somehow', 'else', 'toward', 'me', 'ever', 'within', '’d', 'others', 'same', 'n‘t', 'too', 'ten', '’re', 'make', 'used', 'thereby', 'well', 'any', 'around', '‘m', 'nor', 'always', 'without', 'least', 'thereupon', 'behind', 'show', 'side', 'latterly', 'move', 'wherein', "'m", 'none', 'sometime', 'the', 'really', 'ours', 'made', 'whereupon', 'hence', 'off', 'there', 'besides', 'although', 'often', 'itself', 'hundred', 'does', 'themselves', 'own', '‘d', 'except', 'an', 'mostly', '‘ve', 'both', 'while', 'above', 'beforehand', 'regarding', 'go', "'ve", 'even', 'your', 'throughout', 'namely', 'thi

In [17]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)

    # print(doc)
    # print(type(doc))

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # print(mytokens)

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [19]:
dataset['Tokens'] = dataset['Question_body'].apply(spacy_tokenizer)

In [ ]:
#dataset.head()

In [21]:
#def Bow_vectorizer(word)
x = dataset['Tokens'].map(' '.join)
y = dataset['Label']

In [ ]:
#dataset.head()

# Features Engineering with BoW

In [22]:
X_train, X_test, y_train, y_test = train_test_split(x , y, test_size=0.33, random_state=125)

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(X_train)
X_train_vect = vect.transform(X_train).toarray()
X_test_vect = vect.transform(X_test).toarray()

In [ ]:
#dataset['BoW_vect'] = dataset['Tokens']

# Naive Bayes with BoW

In [23]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

In [24]:
nb_model = GaussianNB()
nb_model = nb_model.fit(X_train_vect, y_train)

In [27]:
#y_pred = nb_model.predict(X_test_vect)
#print(classification_report(y_test, y_pred))

In [26]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Print metrics for each class
precision_class, recall_class, f1_class, _ = precision_recall_fscore_support(y_test, y_pred, average=None, labels=[0, 1])

print(f'Class 0 - Precision: {precision_class[0]}, Recall: {recall_class[0]}, F1-score: {f1_class[0]}')
print(f'Class 1 - Precision: {precision_class[1]}, Recall: {recall_class[1]}, F1-score: {f1_class[1]}')


Class 0 - Precision: 0.8146002317497103, Recall: 0.8290094339622641, F1-score: 0.8217416715371126
Class 1 - Precision: 0.8140230867892262, Recall: 0.7986577181208053, F1-score: 0.8062672030489095


# kNN with BoW

In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [29]:
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(X_train_vect, y_train)

KNeighborsClassifier(n_neighbors=1)

In [ ]:
y_pred = knn_model.predict(X_test_vect)
print(classification_report(y_test, y_pred))

# Decison Tree with BoW

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt_model = DecisionTreeClassifier()
dt_model = dt_model.fit(X_train_vect, y_train)

In [ ]:
y_pred = dt_model.predict(X_test_vect)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95      2544
           1       0.94      0.96      0.95      2384

    accuracy                           0.95      4928
   macro avg       0.95      0.95      0.95      4928
weighted avg       0.95      0.95      0.95      4928



# Logistic Regression with BoW

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lg_model = LogisticRegression(C=5, penalty='l1', solver='liblinear')
lg_model = lg_model.fit(X_train_vect, y_train)

In [ ]:
y_pred = lg_model.predict(X_test_vect)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95      2544
           1       0.94      0.96      0.95      2384

    accuracy                           0.95      4928
   macro avg       0.95      0.95      0.95      4928
weighted avg       0.95      0.95      0.95      4928



#SVM with BoW

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm_model = SVC()
svm_model = svm_model.fit(X_train_vect, y_train)

In [ ]:
y_pred = svm_model.predict(X_test_vect)
print(classification_report(y_test, y_pred))

# Neural Net

In [1]:
import tensorflow as tf

In [2]:
def plot_history(history):
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
  ax1.plot(history.history['loss'], label='loss')
  ax1.plot(history.history['val_loss'], label='val_loss')
  ax1.set_xlabel('Epoch')
  ax1.set_ylabel('Binary crossentropy')
  ax1.grid(True)

  ax2.plot(history.history['accuracy'], label='accuracy')
  ax2.plot(history.history['val_accuracy'], label='val_accuracy')
  ax2.set_xlabel('Epoch')
  ax2.set_ylabel('Accuracy')
  ax2.grid(True)

  plt.show()

In [ ]:
def train_model(X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs):
  nn_model = tf.keras.Sequential([
      tf.keras.layers.Dense(num_nodes, activation='relu', input_shape=(10,)),
      tf.keras.layers.Dropout(dropout_prob),
      tf.keras.layers.Dense(num_nodes, activation='relu'),
      tf.keras.layers.Dropout(dropout_prob),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  nn_model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='binary_crossentropy',
                  metrics=['accuracy'])
  history = nn_model.fit(
    X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0
  )

  return nn_model, history